In [14]:
from IPython import display
import math
from pprint import pprint
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='darkgrid', context='talk', palette='Dark2')
import praw
from praw.models import MoreComments
import spacy
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

In [15]:
spacy.prefer_gpu()
nlp = spacy.load("en_core_web_sm")
sent = "Honestly.. The french just wants it more than other communities. Other streamer communities except for the daliban are just fucking around lol"
doc=nlp(sent)

sub_toks = [tok for tok in doc if (tok.dep_ == "nsubj") ]

print(sub_toks) 

[french, it, communities]


In [16]:
reddit = praw.Reddit(client_id='sRUAAFcF92SAOAcApXtOgA',
                     client_secret='NWHiWc_yM5fCscnAdtDq7HlQKf8U5Q',
                     user_agent='tomtom2352')

In [17]:
def removeWhiteSpace(word):
    return word.strip()

for submission in reddit.subreddit('LivestreamFail').hot(limit=25):
    splitText = submission.link_flair_text.split(":")
    splitText = splitText[len(splitText)-1].split("|")[0].strip()

sia = SIA()
results = []

line = "I don't see dean on my screen this whole convo was hilarious"
pol_score = sia.polarity_scores(line)
pol_score['headline'] = line
results.append(pol_score)

pprint(results[:5], width=100)

sent_text = nltk.sent_tokenize("This is so hilarious I went to see them get into a petty war. After seeing XQC do GTA RP I don't think anybody could be pettier than him")
print(sent_text)


[{'compound': 0.4019,
  'headline': "I don't see dean on my screen this whole convo was hilarious",
  'neg': 0.0,
  'neu': 0.787,
  'pos': 0.213}]
['This is so hilarious I went to see them get into a petty war.', "After seeing XQC do GTA RP I don't think anybody could be pettier than him"]


In [18]:
submission = reddit.submission(id='tt4ar6')
comment = submission.comments[1]
comment.refresh()
replies = comment.replies
print(replies[0].body)

santa deniers 4weird


In [19]:
submission = reddit.submission(id='twanfw')
comments = submission.comments

allComments = comments.list()
for cur in allComments:
    if isinstance(cur, MoreComments):
        continue

In [20]:
titles = []
allComments = []

for submission in reddit.subreddit('LivestreamFail').hot(limit=25):
    splitText = submission.link_flair_text.split(":")
    splitText = splitText[len(splitText)-1].split("|")[0].strip()
    print(splitText)
    
    titles.append(submission.title)
    comments = submission.comments.list()
    for comment in comments[1:]:
        if isinstance(comment, MoreComments):
            continue
        allComments.append(comment.body)
print(allComments)

fanfan
Forsen
Asmongold
narcissawright
Dexerto
CodeMiko
Loud
zachbussey
JakeSucky
CodeMiko
Nmplol
PaymoneyWubby
Nmplol
Quin69
AndyMilonakis
Quin69
twitchgaming
ImKaiCenat
Lacari
AustinShow
AustinShow
MurderCrumpet
Sodapoppin
Lacari
AndyMilonakis
["Now thats what I call 'Standing your ground'", 'and i said ^(biiiiiiitch)', ">This clip's past broadcast has expired and chat replay is no longer available.\n\nTwitch why are you the way that you are", "Don't know who this guy is, but he's funny.", 'Death by a thousand cuts.', "I've never seen such densely packed hair on a person's head before. That's insane.", 'perfect timing lol (the whomp slam of the wallet too hahaha)', 'i guide others to a treasure i cannot possess', 'they both troll each other too much, cant even tell when they are actually serious lol', "Couldn't have been planned better.\n\nThe pout at the end seals it lol.", 'He was Alpha for 20 seconds before he was forced back into his Beta form.  :(', 'xqcTree', 'Wtf is B sauce', 

In [21]:
sia = SIA()
results = []
total = 0
positive = 0
negative = 0
neutral = 0

for line in allComments:
    pol_score = sia.polarity_scores(line)
    pol_score['headline'] = line
    results.append(pol_score)
    total += 1
    if pol_score['compound'] >= 0.05:
        positive += 1
    elif pol_score['compound'] <= -0.05:
        negative += 1
    else:
        neutral += 1

pprint(results[:20], width=100)
print(100*positive/total)
print(100*negative/total)
print(100*neutral/total)

[{'compound': 0.0,
  'headline': "Now thats what I call 'Standing your ground'",
  'neg': 0.0,
  'neu': 1.0,
  'pos': 0.0},
 {'compound': 0.0, 'headline': 'and i said ^(biiiiiiitch)', 'neg': 0.0, 'neu': 1.0, 'pos': 0.0},
 {'compound': -0.296,
  'headline': ">This clip's past broadcast has expired and chat replay is no longer available.\n"
              '\n'
              'Twitch why are you the way that you are',
  'neg': 0.095,
  'neu': 0.905,
  'pos': 0.0},
 {'compound': 0.5927,
  'headline': "Don't know who this guy is, but he's funny.",
  'neg': 0.0,
  'neu': 0.675,
  'pos': 0.325},
 {'compound': -0.7269,
  'headline': 'Death by a thousand cuts.',
  'neg': 0.753,
  'neu': 0.247,
  'pos': 0.0},
 {'compound': -0.4019,
  'headline': "I've never seen such densely packed hair on a person's head before. That's insane.",
  'neg': 0.184,
  'neu': 0.816,
  'pos': 0.0},
 {'compound': 0.5994,
  'headline': 'perfect timing lol (the whomp slam of the wallet too hahaha)',
  'neg': 0.152,
  'neu'

In [22]:
df = pd.DataFrame.from_records(results)
with pd.ExcelWriter('data/learnData.xlsx', mode='a') as writer:  
    df.to_excel(writer, sheet_name='Sheet2')

ValueError: Sheet 'Sheet2' already exists and if_sheet_exists is set to 'error'.

In [ ]:
df.head()

,neg,neu,pos,compound,headline
0,0.000,1.000,0.0,0.00,NL on hitting 1 million YouTube subs
1,0.286,0.714,0.0,-0.34,Stream Sniper comments on Forsen's gun choice
2,0.000,1.000,0.0,0.00,Jerma attempts to land in Fortnite
3,0.000,1.000,0.0,0.00,Based Erobb
4,0.000,1.000,0.0,0.00,Erobb starts WW3
